In [5]:
import torch

print(torch.cuda.is_available())  # 返回 True 表示 CUDA 可用

True


In [ ]:
! pip install -q transformers==4.35.2 einops transformers_stream_generator bitsandbytes datasets accelerate tiktoken gradio

In [11]:
! pip install accelerate bitsandbytes

In [1]:
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

F:\anaconda\Lib\site-packages\transformers\utils\generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


print('''downloading model files ...\nThis step usually takes several minutes.\n''')
! git clone https://www.modelscope.cn/CharPoet/CharPoet.git
print('''\nmodel files downloaded.''')

In [ ]:
quantization_config = BitsAndBytesConfig(load_in_8bit=True)
# model = AutoModelForCausalLM.from_pretrained('./CharPoet', device_map='auto', trust_remote_code=True, quantization_config=quantization_config)
model = AutoModelForCausalLM.from_pretrained('./CharPoet', device_map='auto', trust_remote_code=True, torch_dtype=torch.float16, quantization_config=None)
tokenizer = AutoTokenizer.from_pretrained('./CharPoet', trust_remote_code=True)

F:\anaconda\Lib\site-packages\transformers\utils\generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
F:\anaconda\Lib\site-packages\transformers\utils\generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
F:\anaconda\Lib\site-packages\transformers\utils\generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
from CharPoet.utils import write_poem

In [ ]:
import gradio as gr

demo = gr.Interface(
    fn = lambda prompt, rhythmic: write_poem(model, tokenizer, prompt, rhythmic),
    inputs=[gr.Textbox(label="提示/Prompt", value="请帮我写一首诗，关键词是“春暖花开”")
            ,gr.Radio(["五言绝句/Wuyanjueju", "七言绝句/Qiyanjueju", "五言律诗/Wuyanlvshi", "七言律诗/Qiyanlvshi", "如梦令/Rumengling", "减字木兰花/Jianzimulanhua", "清平乐/Qingpingyue", "蝶恋花/Dielianhua", "满江红/Manjianghong" , "沁园春/Qinyuanchun", "水调歌头/Shuidiaogetou", "菩萨蛮/Pusaman", "卜算子/Busuanzi"], label="诗词种类/Format type", value='七言绝句/Qiyanjueju')
            ],
    outputs=gr.Textbox(label="诗歌创作结果/Result"),
    examples = [
        ["Write me a poem about Spring.", "如梦令/Rumengling"],
        ["Introduce New York city.", "沁园春/Qinyuanchun"]
    ],
    title="CharPoet",
    description="poem generator based on token-free LLM"
)

demo.launch()

In [10]:
import requests

# 替换为你的 Gradio 服务地址（默认是本地 7860 端口）
API_URL = "http://127.0.0.1:7860/api/predict"  

data = {
    "data": [
        "请写一首关于秋天的诗",  # prompt
        "五言绝句/Wuyanjueju"   # rhythmic
    ]
}

response = requests.post(API_URL, json=data)
print(response.json())  # 输出结果会包含诗歌内容

{'detail': 'Not Found'}


In [11]:
pip install gradio_client

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from gradio_client import Client

client = Client("http://127.0.0.1:7860/")
result = client.predict(
		prompt="以“最是人间留不住”为第一句写一首诗",
		rhythmic="七言绝句/Qiyanjueju",
		api_name="/predict"
)
print(result)

Loaded as API: http://127.0.0.1:7860/ ✔


In [1]:
!pip uninstall torch